# Torch Ember Core
> Analyzing How Model Improves

In [1]:
# default_exp core

In this tutorial, we'll use AlexNet as example, 
We can load AlexNet from ```torchvision```

By:
* Lai Wei
* Xiaochen Zhang

In [2]:
from torchvision.models.alexnet import AlexNet
import torch


In [3]:
model = AlexNet()

#### Sample data
Create a sample data, something like 2 normalized images in a batch, size 224,224

In [4]:
samp = (torch.rand(2,3,224,224)-1)*2

In [5]:
model(samp).shape

torch.Size([2, 1000])

## File, log, data, directory handler

In [6]:
# export
import os
from pathlib import Path
import json
import pandas as pd
from datetime import datetime
import torch
from torchember.helper import color
from functools import partial

class tracker(object):
    def __init__(self, libname, fname):
        self.libname = libname
        self.fname = fname
        self.home = Path(os.environ['HOME'])
        self.dir = self.home/f".{libname}"
        self.dir.mkdir(exist_ok = True) 
        self.data = self.dir/"data"
        self.data.mkdir(exist_ok = True) 
        self.log = self.dir/"log"
        self.log.mkdir(exist_ok = True) 
        self.log_path = self.log/self.fname
        
    def __repr__(self):
        return f"<{self.libname}:{self.fname}>"
        
    def mkdir(self, path):
        Path(path).mkdir(exist_ok=True)
        
    def __setitem__(self, fname,dict_):
        with open(self.data/f"{fname}.json","w") as f: f.write(json.dumps(dict_, indent = 2))

    
    def __getitem__(self,fname):
        return json.loads(open(self.data/f"{fname}.json","r").read())
        
    def logging(self,line):
        with open(self.log_path,"a") as f :f.write(line+"\n")
        return self.log_path
        
    def __add__(self,dict_):
        """
        add a dictionary to log
        """
        self.logging(json.dumps(dict_))
        return self
        
    def lines(self):
        return list(json.loads(i) for i in open(self.log_path).read().split("\n")[:-1])
    
    @property
    def ts(self):
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    @property
    def df(self):
        return pd.DataFrame(self.lines())
    
class emberTracker(tracker):
    def __init__(self, fname):
        super().__init__("torchember",fname)
        self.latest = self.log/f"{fname}_latest"
        self.latest_lines = ""
        
    def logging(self,line):
        with open(self.log_path,"a") as f : f.write(line+"\n")
        self.latest_lines+=(line+"\n")
        return self.log_path
    
    def refresh(self):
        """
        lastest always contain the record of the latest batch
        """
        with open(self.latest,"w") as f :  f.write(self.latest_lines)
        self.latest_lines = ""
        return self.latest
    
    def latest_line_list(self):
        return list(json.loads(i) for i in open(self.latest).read().split("\n")[:-1])
    
    @property
    def latest_df(self):
        return pd.DataFrame(self.latest_line_list())
        

### Test on the data tracker

In [7]:
etrack = emberTracker("testmodel")

This is how we record the data within other codes

In [8]:
for i in range(10):
    etrack+{"col1":i,"col2":i*2,"col3":"hahahha"}

Preview log file

In [9]:
!cat {etrack.log_path}

{"col1": 0, "col2": 0, "col3": "hahahha"}
{"col1": 1, "col2": 2, "col3": "hahahha"}
{"col1": 2, "col2": 4, "col3": "hahahha"}
{"col1": 3, "col2": 6, "col3": "hahahha"}
{"col1": 4, "col2": 8, "col3": "hahahha"}
{"col1": 5, "col2": 10, "col3": "hahahha"}
{"col1": 6, "col2": 12, "col3": "hahahha"}
{"col1": 7, "col2": 14, "col3": "hahahha"}
{"col1": 8, "col2": 16, "col3": "hahahha"}
{"col1": 9, "col2": 18, "col3": "hahahha"}


Read log as dataframe

In [10]:
etrack.df

,col1,col2,col3
0,0,0,hahahha
1,1,2,hahahha
2,2,4,hahahha
3,3,6,hahahha
4,4,8,hahahha
5,5,10,hahahha
6,6,12,hahahha
7,7,14,hahahha
8,8,16,hahahha
9,9,18,hahahha


In [11]:
!rm {etrack.log_path}

### Save/ Get JSON data

Save dictionary to json data

In [12]:
etrack["about_this_model"]= {
    "property1":1,
    "property2":False,
    "property3":
        {"size":100,
         "speed":200}
    }

Read this meta data from json file

In [13]:
etrack["about_this_model"]["property2"]

False

## Torch Ember Core

The essence of torch ember, is to place trackers within modules.

It will decorate the ```forward``` function to achieve following purpose

* What variables come in/out of the module
* The happening sequence, containing relationships between sub-modules
* The statistics we want for further analysis, eg.
    * Min, Max, Mean, Std, of input / outpout tensors
    * Min, Max, Mean, Std, of model weights at this iteration
    * Min, Max, Mean, Std, of model weights grad at this iteration

In [16]:
# export
from types import MethodType
from datetime import datetime

class moduleTrack(object):
    def __init__(self,module, name=None, root_module = False):
        self.module = module
        module.module_tracker = self
        
        self.base_module = True if len(list(module.modules()))==1 else False
        self.root_module = root_module
        
        self.name = name if name else module.__class__.__name__  
        #self.name = f'{name}_tracker' if name else f'{module.__class__.__name__}_tracker'
        self.id = id(module)
        self.children = []
        
    def __repr__(self):
        rt = f"<{self.name} @ {hex(self.id)}>"
        if hasattr(self,"input_dt"):
            rt+=f'\n\t[Inputs]{",".join(list(k+" "+str(list(v.shape)) for k,v in self.input_dt.items()))}'
        if hasattr(self,"output_dt"):
            rt+=f'\n\t[Outputs]{",".join(list(str(list(v.shape)) for v in self.output_dt))}'
        return rt

def get_stats(tensor):
    """
    The default statistic method, it will capture
    shape of the tensor
    mean, std, max, min of the tensor
    this will return a dictionary
    """
    return {"shape":list(tensor.shape),
            "mean":tensor.mean().item(), 
            "std":tensor.std().item(), 
            "max":tensor.max().item(), 
            "min":tensor.min().item(),
            "cnt_zero": ((tensor>-1e-10) & (tensor < 1e-10)).sum().item()}


    
class torchEmber(object):
    def __init__(self, model, verbose = True):
        color.green|"start analyzing model"
        self.modules = dict()
        self.verbose = verbose
        self.model = model
        
        if hasattr(model,"disarm"):
            model.disarm()
        
        self.model_name = self.model.__class__.__name__
        
        fname = f"{self.model_name}_{self.ts_str}"
        self.fname = fname
        
        self.t = emberTracker(fname)
        self.current_mt = None
        self.mt_log = []
        
        self.arm()
        
        self.legit_ttypes = ["in","out","weight"]
        for ttype in self.legit_ttypes: self.set_metric(ttype)(get_stats)

        if self.verbose: 
            color.green|f"[INFO][{self.ts_str}]Creating meta data"
        self.t[f"base_{fname}"]={"start":self.t.ts, 
                                 "user":os.environ["USER"]}
        self.t[f"vis_{fname}"] = {"vis_type":"standard"}
        self.t[f"structure_{fname}"] = self.mod_tree()
        
    def parse_module(self,model, name, root_module = False):
        name = f"{name}({model.__class__.__name__})"
        mt = moduleTrack(model, name, root_module)
        self.modules[name]= mt
        model.forward = self.module_register(name,model)
        
        for cname,children in model.named_children():
            children_mt = self.parse_module(children,f"{name}.{cname}" )
            children_mt.parent = mt
            mt.children.append(children_mt)
        return mt
    
    def mod_tree(self):
        """
        Return the tree of module
        """
        return self.mod_tree_parse(self.model.module_tracker)
        
    def mod_tree_parse(self,mt):
        rt = {"name":mt.name, "short":mt.name.split(".")[-1]}
        if len(mt.children)>0:
            rt.update({"children":list(self.mod_tree_parse(i) for i in mt.children)})
        return rt
                
        
    @property
    def ts_str(self):
        return datetime.now().strftime("%Y%m%d_%H%M%S")
    
    @property
    def ts(self):
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    def arm(self):
        """
        arming the tracing function to self.model
        """
        if self.verbose: 
            color.yellow|f"[ARMING][START]{self.ts}"
        self.parse_module(self.model,"model", root_module = True)
        if self.verbose: 
            color.yellow|f"[ARMING][SUCCESS]{self.ts}"
            
    def disarm(self):
        """remove the tracing function"""
        for m in self.modules.values():
            if self.verbose: 
                color.blue|f"[DISARM][{m.name}]{self.ts}"
            self.recover(m)
        color.blue|f"[DISARM][DONE]{self.ts}"
            
    def recover(self, m):
        if hasattr(m.module.forward,"former"):
            m.module.forward = m.module.forward.former
            
    def rearm(self):
        self.disarm()
        self.arm()
    
    def reg_check(self,m):
        """
        register check
        """
        if hasattr(m.forward,"armed"):
            if m.forward.armed:
                return False
        return True
    
    def set_metric(self, ttype):
        assert ttype in self.legit_ttypes, f"ttype has to be one of {str(self.ttypes)}"
        def deco(f):
            setattr(self,f"record_{ttype}_core",self.record_core(f))
            return f
        return deco
    
    def add_record(f):
        def _inner(self, f_name): return partial(f, self, f_name)
        return _inner
    
    @add_record
    def record_core(self, f_name, tensor, extra_data):
        """
        extra_data: dict
        """
        dict_= f_name(tensor)
        dict_.update(extra_data)
        self.t+ dict_
        return dict_
    
    def record_input(self,mt):
        """
        Record the input tensors of the moduleTrack
        """
        for k,tensor in mt.input_dt.items():
            self.record_in_core(tensor,
                          extra_data = {"module":mt.name,"ts":self.t.ts,"ttype":"input","tname":k})
            
    def record_output(self,mt):
        """
        Record the output tensors of the moduleTrack
        """
        for i in range(len(mt.output_dt)):
            tensor = mt.output_dt[i]
            self.record_out_core(tensor,
                          extra_data = {"module":mt.name,"ts":self.t.ts,"ttype":"output","tname":f"output_{i}"})
            
    def record_weight(self,mt):
        """
        Record the weights of the moduleTrack
        """
        if mt.base_module:
            i = 0
            for p in mt.module.parameters():
                self.record_weight_core(p.data, 
                              extra_data = {"module":mt.name,"ts":self.t.ts,
                                            "ttype":"weight","tname":f"weight_{i}"})
                if p.requires_grad and (p.grad!= None) :
                    self.record_weight_core(p.grad, 
                              extra_data = {"module":mt.name,"ts":self.t.ts,
                                            "ttype":"weight_grad","tname":f"grad_{i}"})
                i+=1
            
    def module_register(self,name,m):
        if self.reg_check(m) == False: return m.forward
        f = m.forward
        mt = self.modules[name]
        vs = f.__code__.co_varnames
        mt.vars = vs[1:]
        if self.verbose: 
            color.cyan | f"[BUILD FORWARD][{name}]{self.ts}"
        def new_forward(*args,**kwargs):
            mt.input_dt = dict(zip(mt.vars[:len(args)],args))
            mt.input_dt.update(kwargs)
            
            self.record_input(mt)
            self.current_mt = mt
            if mt.root_module: self.mt_log=[]
            self.mt_log.append(f"enter {mt.name}")
            
            # ------execution of the function------
            outputs = f(*args,**kwargs)
            self.record_weight(mt)
            # ------execution of the function------
            
            self.mt_log.append(f"exit {mt.name}")
            
            if type(outputs) in [list,tuple]:
                mt.output_dt = [outputs]
            else:
                mt.output_dt = [outputs,]
            self.record_output(mt)
            
            if mt.root_module:
                self.t.refresh() # start a new "latest" file
            
            return outputs
        
        setattr(new_forward,"armed",True)
        setattr(new_forward,"former",f)
        
        def disarm(this):
            """
            Remove the trackers placed by torchember
            run model.disarm()
            """
            self.disarm()
            return this
        setattr(mt.module, "disarm",MethodType(disarm,mt.module))
        return new_forward

### Tracking a model !!

Start tracking a model

In [17]:
te = torchEmber(model)

start analyzing model
[DISARM][model(AlexNet)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 22:46:54
[DISARM][model(AlexNet).features(Sequential).

Remove the trackers we placed

In [18]:
model = model.disarm()

[DISARM][model(AlexNet)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 22:46:58
[DISARM][model(AlexNet).features(Sequential).11(ReLU)]2020-02-24 22

Or like this

In [19]:
te.disarm()

[DISARM][model(AlexNet)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 22:47:00
[DISARM][model(AlexNet).features(Sequential).11(ReLU)]2020-02-24 22

Okay, refresh the tracker

In [20]:
te.rearm()

[DISARM][model(AlexNet)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 22:47:02
[DISARM][model(AlexNet).features(Sequential).11(ReLU)]2020-02-24 22

Run forward pass for 3 iterations, nothing strange happend

In [21]:
for i in range(3):
    model(samp)

### Check snowballing tensor stats

In [22]:
te.t.df

,shape,mean,std,max,min,cnt_zero,module,ts,ttype,tname
0,"[2, 3, 224, 224]",-1.000480,0.577501,-0.000004,-1.999992,0,model(AlexNet),2020-02-24 22:47:04,input,x
1,"[2, 3, 224, 224]",-1.000480,0.577501,-0.000004,-1.999992,0,model(AlexNet).features(Sequential),2020-02-24 22:47:04,input,input
2,"[2, 3, 224, 224]",-1.000480,0.577501,-0.000004,-1.999992,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:04,input,input
3,"[64, 3, 11, 11]",-0.000011,0.030162,0.052484,-0.052475,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:04,weight,weight_0
4,[64],-0.002946,0.033981,0.050660,-0.052249,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:04,weight,weight_1
...,...,...,...,...,...,...,...,...,...,...
187,"[1000, 4096]",-0.000003,0.009020,0.015625,-0.015625,1,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:47:06,weight,weight_0
188,[1000],-0.000137,0.008945,0.015592,-0.015614,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:47:06,weight,weight_1
189,"[2, 1000]",-0.000115,0.011572,0.037493,-0.043643,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:47:06,output,output_0
190,"[2, 1000]",-0.000115,0.011572,0.037493,-0.043643,0,model(AlexNet).classifier(Sequential),2020-02-24 22:47:06,output,output_0


Now let's start record weight grad data, once we use backward(), we'll soon have grad data kick in when next forward pass is called

### Track weight gradients

In [23]:
for i in range(3):
    model(samp).mean().backward()

As you can see here, for conv layer, 
* grad_0 is for the 1st weight grad tensor(weight), 
* grad_1 is for the 2nd(bias)

In [24]:
te.t.df[te.t.df["ttype"]=="weight_grad"].head()

,shape,mean,std,max,min,cnt_zero,module,ts,ttype,tname
260,"[64, 3, 11, 11]",2.473037e-07,0.000024,0.000094,-0.000094,363,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:19,weight_grad,grad_0
262,[64],-2.520948e-07,0.000023,0.000063,-0.000058,1,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:19,weight_grad,grad_1
270,"[192, 64, 5, 5]",-7.922446e-08,0.000016,0.000111,-0.000154,35086,model(AlexNet).features(Sequential).3(Conv2d),2020-02-24 22:47:19,weight_grad,grad_0
272,[192],9.021236e-08,0.000021,0.000055,-0.000088,4,model(AlexNet).features(Sequential).3(Conv2d),2020-02-24 22:47:19,weight_grad,grad_1
280,"[384, 192, 3, 3]",-2.164734e-07,0.000012,0.000171,-0.000122,162009,model(AlexNet).features(Sequential).6(Conv2d),2020-02-24 22:47:19,weight_grad,grad_0


### Module tree json
This file will be stored at ```$HOME/.torchember/data/structure_<modelname>_<date>_<time>.json```

In [25]:
te.mod_tree()

{'name': 'model(AlexNet)',
 'short': 'model(AlexNet)',
 'children': [{'name': 'model(AlexNet).features(Sequential)',
   'short': 'features(Sequential)',
   'children': [{'name': 'model(AlexNet).features(Sequential).0(Conv2d)',
     'short': '0(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).1(ReLU)',
     'short': '1(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).2(MaxPool2d)',
     'short': '2(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).3(Conv2d)',
     'short': '3(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).4(ReLU)',
     'short': '4(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).5(MaxPool2d)',
     'short': '5(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).6(Conv2d)',
     'short': '6(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).7(ReLU)',
     'short': '7(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).8(Conv2d)',
     'short': '8(Conv2d)'},
    {'name': 'model(AlexN

In [26]:
te.mt_log

['enter model(AlexNet)',
 'enter model(AlexNet).features(Sequential)',
 'enter model(AlexNet).features(Sequential).0(Conv2d)',
 'exit model(AlexNet).features(Sequential).0(Conv2d)',
 'enter model(AlexNet).features(Sequential).1(ReLU)',
 'exit model(AlexNet).features(Sequential).1(ReLU)',
 'enter model(AlexNet).features(Sequential).2(MaxPool2d)',
 'exit model(AlexNet).features(Sequential).2(MaxPool2d)',
 'enter model(AlexNet).features(Sequential).3(Conv2d)',
 'exit model(AlexNet).features(Sequential).3(Conv2d)',
 'enter model(AlexNet).features(Sequential).4(ReLU)',
 'exit model(AlexNet).features(Sequential).4(ReLU)',
 'enter model(AlexNet).features(Sequential).5(MaxPool2d)',
 'exit model(AlexNet).features(Sequential).5(MaxPool2d)',
 'enter model(AlexNet).features(Sequential).6(Conv2d)',
 'exit model(AlexNet).features(Sequential).6(Conv2d)',
 'enter model(AlexNet).features(Sequential).7(ReLU)',
 'exit model(AlexNet).features(Sequential).7(ReLU)',
 'enter model(AlexNet).features(Sequentia

### Check latest tensor stats

In [27]:
te.t.latest_df

,shape,mean,std,max,min,cnt_zero,module,ts,ttype,tname
0,"[2, 3, 224, 224]",-1.000480e+00,0.577501,-0.000004,-1.999992,0,model(AlexNet),2020-02-24 22:47:21,input,x
1,"[2, 3, 224, 224]",-1.000480e+00,0.577501,-0.000004,-1.999992,0,model(AlexNet).features(Sequential),2020-02-24 22:47:21,input,input
2,"[2, 3, 224, 224]",-1.000480e+00,0.577501,-0.000004,-1.999992,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:21,input,input
3,"[64, 3, 11, 11]",-1.124644e-05,0.030162,0.052484,-0.052475,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:21,weight,weight_0
4,"[64, 3, 11, 11]",7.861846e-07,0.000038,0.000150,-0.000118,363,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:47:21,weight_grad,grad_0
...,...,...,...,...,...,...,...,...,...,...
75,[1000],-1.374600e-04,0.008945,0.015592,-0.015614,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:47:22,weight,weight_1
76,[1000],2.000000e-03,0.000000,0.002000,0.002000,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:47:22,weight_grad,grad_1
77,"[2, 1000]",-3.248150e-04,0.011494,0.040849,-0.037424,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:47:22,output,output_0
78,"[2, 1000]",-3.248150e-04,0.011494,0.040849,-0.037424,0,model(AlexNet).classifier(Sequential),2020-02-24 22:47:22,output,output_0


### Redifine what you want to record

For the default statistic function, you can keep track shape, mean, std, max,min of a tensor.

The afore-mentioned tensor could mean all of the following
* module input tensors
* module output tensors
* module weight
* gradient of module weight

If you have more interesting metrics to follow, you can redifine the statistic tracking function

#### Redifine the weight tensor/ weight grad tensor  statitic function

In [28]:
@te.set_metric("weight")
def weight_stats(tensor):
    return {"num":tensor.numel(),"row_max":list(row.max().item() for row in tensor)}

#### Redifine the input or output statitic function

In [29]:
@te.set_metric("in")
def input_stats(tensor):
    return {"num":tensor.numel(),"row_min":list(row.min().item() for row in tensor)}

@te.set_metric("out")
def output_stats(tensor):
    return {"num":tensor.numel(),"row_min":list(row.min().item() for row in tensor)}

Let's give 1 forward pass again

In [30]:
model(samp)

tensor([[ 1.2442e-02,  2.3200e-04,  6.2544e-03,  ...,  1.4764e-02,
          1.2834e-03,  5.8982e-03],
        [ 1.7643e-02,  6.4590e-03, -2.7058e-05,  ...,  3.5622e-03,
         -1.1052e-03, -1.9941e-03]], grad_fn=<AddmmBackward>)

The latest stats changed

In [31]:
te.t.latest_df

,num,row_min,module,ts,ttype,tname,row_max
0,301056,"[-1.999979019165039, -1.999991774559021]",model(AlexNet),2020-02-24 22:48:06,input,x,NaN
1,301056,"[-1.999979019165039, -1.999991774559021]",model(AlexNet).features(Sequential),2020-02-24 22:48:06,input,input,NaN
2,301056,"[-1.999979019165039, -1.999991774559021]",model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:48:06,input,input,NaN
3,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:48:06,weight,weight_0,"[0.05245126038789749, 0.0521940216422081, 0.05..."
4,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 22:48:06,weight_grad,grad_0,"[8.217657523346134e-07, 4.583279223879799e-05,..."
...,...,...,...,...,...,...,...
75,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:48:07,weight,weight_1,"[0.013463851064443588, -0.0014015156775712967,..."
76,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:48:07,weight_grad,grad_1,"[0.003000000026077032, 0.003000000026077032, 0..."
77,2000,"[-0.032634783536195755, -0.03472833335399628]",model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 22:48:07,output,output_0,NaN
78,2000,"[-0.032634783536195755, -0.03472833335399628]",model(AlexNet).classifier(Sequential),2020-02-24 22:48:07,output,output_0,NaN


## Placing tracker on variables
To be experimented here

In [32]:
w = list(model.features.parameters())[0]

In [33]:
from types import BuiltinMethodType,BuiltinFunctionType

In [34]:
x1 = torch.rand(5,6)
x2 = torch.rand(5,6)
x3 = x1*6+x2

In [35]:
x2.numel()

30

In [36]:
x1.abs_()

tensor([[0.2660, 0.9816, 0.9402, 0.9817, 0.0382, 0.7806],
        [0.1407, 0.5267, 0.8065, 0.2384, 0.5621, 0.6195],
        [0.5146, 0.7612, 0.1067, 0.1149, 0.8720, 0.2196],
        [0.3673, 0.4999, 0.9720, 0.8592, 0.3099, 0.2564],
        [0.3932, 0.7050, 0.1794, 0.2110, 0.4561, 0.9326]])

In [37]:
from types import MethodType

In [38]:
import inspect

In [39]:
def TorchTensorEmber(x):
    class TensorEmber(x.__class__):
        def __init__(self,x):
            self.host_ = x
            attrs = dir(x)
            for attr in attrs:
                self.super_attr(attr)
            
        def super_attr(self,attr):
            if inspect.isbuiltin(getattr(self.host_,attr))==False: return 
            def func(self,*args,**kwargs):
                print(attr)
                return getattr(super(),attr)(*args,**kwargs)
            func.__name__ = attr
            setattr(self,attr, MethodType(func,self))
            return func
            
    return TensorEmber(x)

In [40]:
x2 = TorchTensorEmber(x2)

In [41]:
x2.add(x1)

add


tensor([[0.9676, 1.0208, 1.0302, 1.8530, 0.4761, 1.5738],
        [0.2713, 1.3983, 1.7669, 0.5223, 0.7321, 1.4964],
        [0.7421, 1.5497, 0.8475, 0.9706, 1.1351, 1.0011],
        [0.7293, 1.2904, 1.6976, 1.6464, 0.3426, 0.6262],
        [0.7965, 1.2458, 0.7171, 0.7595, 0.6588, 1.0484]])

In [42]:
x2+x1

tensor([[0.9676, 1.0208, 1.0302, 1.8530, 0.4761, 1.5738],
        [0.2713, 1.3983, 1.7669, 0.5223, 0.7321, 1.4964],
        [0.7421, 1.5497, 0.8475, 0.9706, 1.1351, 1.0011],
        [0.7293, 1.2904, 1.6976, 1.6464, 0.3426, 0.6262],
        [0.7965, 1.2458, 0.7171, 0.7595, 0.6588, 1.0484]])

## Placing tracker on optimizer
To be experimented here